# ПРОГНОЗ ОТТОКА ПОЛЬЗОВАТЕЛЕЙ КРЕДИТНЫХ КАРТ

## ЗАДАЧА
Необходимо построить прогнозную модель, которая сможет определить, перестанет ли клиент банка пользоваться кредитной картой.

#### В таблице представлены следующие столбцы:

**CLIENTNUM** - Номер клиента. Уникальный идентификатор клиента, которому принадлежит счет

**Attrition_Flag** - Aктивность клиента — если учетная запись закрыта, то Attrited Customer, иначе Existing Customer

**Customer_Age** - Bозраст клиента в годах

**Gender** - М=мужчина, Ж=женщина

**Dependent_count** - Количество иждивенцев

**Education_Level** - Образование, квалификация владельца счета (пример: средняя школа, выпускник колледжа и т. д.)

**Marital_Status** - Женат, Холост, Разведен, Неизвестно

**Income_Category** - Категория годового дохода владельца счета (< 40 000 долларов США, 40 000-60 000 долларов США, 60 000-80 000 долларов США, 80 000-120 000 долларов США, >

**Card_Category** - Тип карты (Синяя, Серебряная, Золотая, Платиновая)

**Months_on_book** - Период отношений с банком

**Total_Relationship_Count** - Общее количество продукции, хранящейся у клиента

**Months_Inactive_12_mon** - Количество месяцев бездействия за последние 12 месяцев

**Contacts_Count_12_mon** - Количество контактов за последние 12 месяцев

**Credit_Limit** - Кредитный лимит по кредитной карте

**Total_Revolving_Bal** - Общий оборотный баланс на кредитной карте

**Avg_Open_To_Buy** - Открытая кредитная линия для покупки (среднее значение за последние 12 месяцев)

**Total_Amt_Chng_Q4_Q1** - Изменение суммы транзакции (Q4 по сравнению с Q1)

**Total_Trans_Amt** - Общая сумма транзакции (последние 12 месяцев)

**Total_Trans_Ct** - Общее количество транзакций (последние 12 месяцев)

**Total_Ct_Chng_Q4_Q1** - Изменение количества транзакций (Q4 по сравнению с Q1)

**Avg_Utilization_Ratio** - Средний коэффициент использования карт

### Целевой признак
**Attrition_Flag** - Aктивность клиента — если учетная запись закрыта, то Attrited Customer, иначе Existing Customer

**МЕТРИКА**

Вычислите оценку F1, также известную как сбалансированная F-оценка или F-мера.

Показатель F1 можно интерпретировать как среднее гармоническое между точностью и отзывом, где показатель F1 достигает своего наилучшего значения при 1, а наихудший показатель при 0. Относительный вклад точности и отзыва в показатель F1 равен.

Формула для оценки F1:

F1 = 2 * (precision * recall) / (precision + recall)

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torchmetrics import F1Score
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

In [ ]:
train_df = pd.read_csv('train.csv')

In [3]:
validation_df = pd.read_csv('validation.csv')

In [4]:
train_df.head(3)

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,715630983,31,F,0,Graduate,Single,Less than $40K,Blue,21,4,...,3,4598.0,0,4598.0,0.439,6317,77,0.833,0.000,Attrited Customer
1,713643858,53,F,1,College,Married,$40K - $60K,Blue,43,4,...,2,1525.0,1411,114.0,0.660,1911,47,0.958,0.925,Existing Customer
2,708334158,46,F,3,Graduate,Divorced,Less than $40K,Blue,34,4,...,4,9863.0,686,9177.0,0.581,3068,58,0.933,0.070,Existing Customer


In [5]:
numeric_features = []
object_features = []
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        object_features.append(col)
    else:
        numeric_features.append(col)

Заменим значения целевого признака на:

    1 - Attrited Customer
    0 - Existing Customer

In [6]:
train_df['Attrition_Flag'] = train_df['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)

In [7]:
df = pd.concat([train_df[numeric_features], (validation_df[numeric_features])])

In [8]:
scaler = MinMaxScaler()

In [9]:
X_scaler = df[numeric_features].iloc[:, 1:].values

In [10]:
X = train_df[numeric_features].iloc[:, 1:].values

In [11]:
y = train_df['Attrition_Flag'].values

In [12]:
scaler.fit(X_scaler)

MinMaxScaler()

In [13]:
X = scaler.transform(X)

In [14]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [15]:
class MyDataset(Dataset):
    def __init__(self, X, y=None):
        self.n_samples = X.shape[0]
        self.x_data = torch.from_numpy(X).type(torch.float32)
        if not y is None:
            self.y_data = torch.from_numpy(y).type(torch.int64)
        else:
            self.y_data = torch.zeros(self.n_samples).type(torch.float)

    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [16]:
# Класс - модуль, реализующий архитектуру. Можно было бы использовать nn.Sequential
class NeuralNetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetModel, self).__init__()
        # Первый полносвязный слой
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim, device=device) 

        # Нелинейная функция
        self.activation = torch.nn.ReLU()
    
        
        # Полносвязный слой
#         self.fc2 = torch.nn.Linear(hidden_dim, output_dim, device=device)  
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim, device=device)
        self.fc3 = torch.nn.Linear(hidden_dim, hidden_dim, device=device)
        self.fc4 = torch.nn.Linear(hidden_dim, output_dim, device=device)
        
    def forward(self, x):
        # Полносвязный слой
        out = self.fc1(x)

#         # Нелинейная функция
#         out = self.activation(out)
        out = self.activation(out)
        out = self.fc2(out)
        out = self.activation(out)
        out = self.fc3(out)
        out = self.activation(out)
        out = self.fc4(out)

        # Полносвязный слой
#         out = self.fc2(out)
        
        return out

In [17]:
batch_size = 100
n_iters = 200000

In [18]:
input_dim = X.shape[1]
hidden_dim = 100
output_dim = 2

In [19]:
criterion = torch.nn.CrossEntropyLoss()
f1 = F1Score(num_classes=2).to(device)

In [20]:
learning_rate = 0.1

In [21]:
model = NeuralNetModel(input_dim, hidden_dim, output_dim)

In [22]:
model.to(device)

NeuralNetModel(
  (fc1): Linear(in_features=14, out_features=100, bias=True)
  (activation): ReLU()
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=2, bias=True)
)

Обучим нашу сеть

In [23]:
# Передаем параметры в оптимизатор, чтобы для них считались градиенты
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)#, random_state=43)

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [25]:
iteration = 0
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):

        inputs = inputs.cuda(device)
        labels = labels.reshape(-1)
        labels = labels.cuda(device)

        # Затираем прошлые градиенты, чтобы они не аккумулировались
        optimizer.zero_grad()

        # Получаем выход модели
        outputs = model(inputs)

        # Вычисляем значение функции потерь
        loss = criterion(outputs, labels)


        # Здесь происходит вычисление градентов для параметров модели, которые переданы в оптимизатор
        loss.backward()

        # Здесь происходит обновление параметров
        optimizer.step()

        iteration += 1

        if iteration % 20000 == 0:
            # Вычисляем точность (можно пользоваться и sklearn)         
            correct = 0
            total = 0
            # Итерация по всему валидационному датасету, "переключаем" модель в режим инференса
            # (в данном случае не обязательно, но в общем случае важно не забывать)
            model.eval()

            # Поскольку нам не нужно вычисление градиентов, мы используем менеджер контекста
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs = inputs.cuda(device)
                    labels = labels.reshape(-1)
                    labels = labels.cuda(device)

                    # Получаем выход модели
                    outputs = model(inputs)

                    # Берем класс, имеющий наибольшую вероятность (здесь логиты)
                    _, predicted = torch.max(outputs.data, 1)

                    # Подсчитываем общее количество объектов в тестовой выборке
                    total += labels.size(0)

                    # Общее количество верно классифицированных объектов
                    correct += f1(predicted, labels)
                model.train()
            f1_score = 100 * correct / total

            print('Итерация: {}. Validation loss: {}. Validation f1_score: {}'.format(iteration, loss.item(), f1_score))
print('STOP')

Итерация: 20000. Validation loss: 0.02863016165792942. Validation f1_score: 0.9557289481163025
Итерация: 40000. Validation loss: 0.002293114550411701. Validation f1_score: 0.9546757340431213
Итерация: 60000. Validation loss: 0.0002579888969194144. Validation f1_score: 0.9525639414787292
Итерация: 80000. Validation loss: 0.00010038055188488215. Validation f1_score: 0.953090488910675
Итерация: 100000. Validation loss: 6.272739119594917e-05. Validation f1_score: 0.9541437029838562
Итерация: 120000. Validation loss: 7.731480582151562e-05. Validation f1_score: 0.9536170959472656
Итерация: 140000. Validation loss: 2.2534062736667693e-05. Validation f1_score: 0.9536170959472656
Итерация: 160000. Validation loss: 1.49766447066213e-05. Validation f1_score: 0.9536170959472656
Итерация: 180000. Validation loss: 4.2555890104267746e-05. Validation f1_score: 0.9536170959472656
Итерация: 200000. Validation loss: 3.603280129027553e-05. Validation f1_score: 0.9536170959472656
STOP


In [26]:
torch.save(model.state_dict(), 'model_200.pt')

In [27]:
X_valid = validation_df[numeric_features].iloc[:, 1:].values
X_valid = scaler.transform(X_valid)

validation_dataset = MyDataset(X_valid)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                           batch_size=len(validation_dataset), 
                                           shuffle=False)

for inputs, labels in validation_loader:
    inputs = inputs.cuda(device)
    labels = labels.reshape(-1)
    labels = labels.cuda(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

In [28]:
predicted = predicted.cpu()
predicted_array = predicted.numpy()
client_id_array = validation_df.CLIENTNUM.values

df_result = pd.DataFrame(np.hstack([client_id_array.reshape(-1, 1), predicted_array.reshape(-1, 1)]), columns=['client_id', 'churn'])
df_result.to_csv('7_result_evstratovsv_200.csv', index=False)